# Data Mining (KEN4113) #
## Lab 1: Regression ##

Konstantin Sandfort
i6255681

In [1]:
# Imports
import numpy as np

## 1. Linear Regression Model Analysis ##

### 1.1 Gender Bias Analysis

In [31]:
b_0 = 50
b_1 = 20
b_2 = 0.07
b_3 = 10
b_4 = 0.01
b_5 = -3

b = [b_0, b_1, b_2, b_3, b_4, b_5]

# Example of male and female person after graduation

male_01 = [2, 100, 1]  # GPA, IQ, Gender Code
female_01 = [2, 100, 0]

def calc_salary(person, coeff):
    salary = coeff[0] + (person[0] * coeff[1]) + (person[1] * coeff[2]) + (person[2] * coeff[3])
    return salary

# Generate 1000 male and female individuals with random GPA and IQ
# We don't know anything about the distribution, so we assume a normal distribution
n = 10
# First, random numbers from a uniform distribution are generated
male_gpa = np.random.normal(2, 1, n)
male_iq = np.random.normal(100, 15, n)

female_gpa = np.random.normal(2, 1, n)
female_iq = np.random.normal(100, 15, n)

# Adjust values out of range
def squish_to_bounds(x, lower, upper):
    if x < lower:
        return lower
    elif x > upper:
        return upper
    else:
        return x

squish_vec = np.vectorize(squish_to_bounds)

male_gpa_adjusted = squish_vec(male_gpa, 0, 4)
male_iq_adjusted = squish_vec(male_iq, 70, 130)

female_gpa_adjusted = squish_vec(female_gpa, 0, 4)
female_iq_adjusted = squish_vec(female_iq, 70, 130)

males = np.transpose(np.vstack([male_gpa_adjusted, male_iq_adjusted, np.ones(n)]))
females = np.transpose(np.vstack([female_gpa_adjusted, female_iq_adjusted, np.zeros(n)]))
print(males)
print(males.shape)

[[  2.53118903  94.76420479   1.        ]
 [  1.1967939   85.35111632   1.        ]
 [  1.61752134 110.31140771   1.        ]
 [  1.64573726  87.24359      1.        ]
 [  1.27668199 114.13697575   1.        ]
 [  1.38834924 117.91286365   1.        ]
 [  2.18546248 105.74324101   1.        ]
 [  1.6740092  117.48800503   1.        ]
 [  2.38491823 105.75956519   1.        ]
 [  0.82565468 107.20196791   1.        ]]
(10, 3)
